In [1]:
# Импорт необходимых библиотек

import csv
import pandas as pd
from sklearn.cluster import MeanShift
import numpy as np
import collections
import scipy

__Задание:__ Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Надо выбрать 20 таких локаций для размещения, чтобы польза была большой.
Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры.Также эти места должны быть популярны среди туристов.
Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

In [2]:
# Часть открытых данных была скачана в формате dat, для дальнейшей работы преобразуем данные
# в формат csv с разделением по "|" и удалением строк с недостающими данными

with open('checkins.dat') as datfile:
    with open('checkins2.csv','w') as bannerfile:
        csv_writer = csv.writer(bannerfile)
        i=0
        for line in datfile:
            row = [field.strip() for field in line.split('|')]
            if len(row) == 6 and row[3] and row[4]:
                csv_writer.writerow(row)
                i+=1

In [3]:
# Откроем преобразованный файл на чтение и отобразим первые 5 строк.
data=pd.read_csv('checkins2.csv', sep=',')
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [4]:
# Далее посмотрим на информацию о данных с уточнением об использовании памяти.
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396634 entries, 0 to 396633
Data columns (total 6 columns):
id            396634 non-null int64
user_id       396634 non-null int64
venue_id      396634 non-null int64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(2), int64(3), object(1)
memory usage: 39.3 MB


In [5]:
# Согласно информации выше, пропущенных данных в файле нет, но на всякий случай убедимся в этом.
data.isnull().values.any()

False

In [6]:
# Изменим формат данных для уменьшения необходимой памяти.

def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b/1024**2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

data_float = data.select_dtypes(include=['float'])
converted_float = data_float.apply(pd.to_numeric, downcast='float')

print (mem_usage(data_float))
print (mem_usage(converted_float))

compare_floats = pd.concat([data_float.dtypes, converted_float.dtypes], axis=1)
compare_floats.columns = ['before', 'after']
compare_floats.apply(pd.Series.value_counts)

6.00 MB
3.00 MB


,before,after
float32,NaN,2.0
float64,2.0,NaN


In [8]:
# Для выбора локаций, нас интересуют колонки с координатами (широта и долгота).

columns=['latitude','longitude']
datacruise=converted_float[columns]

In [10]:
# Чтобы выявить кластеры, воспользуемся алгоритмом MeanShift, указав ограничение 
# для размера кластеров bandwidth=0.1 (примерно от 5 до 10 км).

ms = MeanShift(bandwidth = 0.1)
ms.fit(datacruise) #обучим алгоритм на выбранных данных

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

In [11]:
# Посмотрим сколько всего кластеров получилось, обозначим уникальные кластеры и их центры.

labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters = len(labels_unique)
print 'Количество кластеров: {}'.format(n_clusters)

Количество кластеров: 5536


In [12]:
# Посчитаем количество элементов в каждом кластере и преобразуем в словарь.

c = collections.Counter(labels)
labelnew = dict(c)

# Соединим в одну матрицу 4 столбца: номер кластера, количество элементов в нем, широта и долгота.
L = np.zeros((3230,4))
i = 0
for line in cluster_centers:
    a = labelnew[i]
    L[i:] = (int(i),a,cluster_centers[i,0],cluster_centers[i,1])
    i+=1

Согласно пояснению: "Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения."

In [13]:
# Оставим кластеры, в которых более 15-ти элементов. И посмотрим сколько таких кластеров.

K = np.zeros((sum(L[:,1]>15),4))
j=0
for i in range(0,len(L)):
    p = L[i,1]
    if p>15:        
        K[j:] = (L[i,0],L[i,1],L[i,2],L[i,3])
        j+=1
    i+=1    
    
print len(K)

1301


Согласно дополнению: "При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров".       

In [14]:
# Запишем в файл csv получившиеся центры кластеров.
        
with open('formap.csv','w') as mapp:
    for i in range(0,len(K)):
        line = ','.join([str(K[i,2]),str(K[i,3])])
        mapp.write(line+'\n')
        i+=1

In [15]:
# По условию 20 баннеров надо разместить близ офисов компании. Координаты офисов также даны.
# Посчитаем расстояния от данных офисов до каждого центра полученных кластеров.

# координаты офисов:

offices = [[33.751277,-118.188740, 'Los Angeles'],
          [25.867736,-80.324116, 'Miami'],
          [51.503016,-0.075479, 'London'],
          [52.378894,4.885084, 'Amsterdam'],
          [39.366487,117.036146, 'Beijing'],
          [-33.868457, 151.205134,'Sydney']]

Distance = np.zeros((len(K)*6,4))
k = 0
for i in range(0,len(K)):
    for j in range(0,len(offices)):
        dist = scipy.spatial.distance.euclidean(offices[j][:2], K[i,2:4])
        Distance[k:] = (K[i,0],dist,K[i,2],K[i,3])
        j+=1
        k+=1
i+=1

In [16]:
# Отсортируем кластеры по удаленности от офисов. И первые 20 баннеров как раз и нужно будет разместить, 
# согласно указанным координатам.

Distance = pd.DataFrame(Distance)
Distance_sorted = Distance.rename(columns={0:'№ of cluster', 1:'Distance', 2:'Latitude', 3:'Longitude'}).sort_values('Distance')
Distance_sorted.head(20)

,№ of cluster,Distance,Latitude,Longitude
1511,251.0,0.003023,-33.866146,151.207082
1917,319.0,0.009625,52.372489,4.892268
1903,317.0,0.025084,25.846206,-80.311245
332,55.0,0.051634,51.503055,-0.127113
288,48.0,0.074644,33.811275,-118.144334
145,24.0,0.135583,25.787086,-80.215128
589,98.0,0.181390,26.005052,-80.205598
474,79.0,0.194084,33.898488,-118.062259
5090,849.0,0.233944,51.480366,-0.308323
5780,964.0,0.264100,51.598314,-0.321786
